<a href="https://colab.research.google.com/github/parul33/Deeplearning/blob/main/TransferLearning12sep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
 import tensorflow as tf
 from tensorflow.keras import layers, models
 from tensorflow.keras.applications import VGG16
 from tensorflow.keras.datasets import mnist
 from tensorflow.keras.utils import to_categorical


In [11]:


 (train_images, train_labels), (test_images, test_labels) = mnist.load_data()


In [5]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Resize images to 32x32 (as VGG16 requires at least 32x32) and keep the last dimension as 1
train_images_resized = tf.image.resize(train_images, (32, 32))
test_images_resized = tf.image.resize(test_images, (32, 32))

# Since VGG16/MobileNet requires 3-channel input, replicate the grayscale image across 3 channels
train_images_rgb = tf.image.grayscale_to_rgb(train_images_resized)
test_images_rgb = tf.image.grayscale_to_rgb(test_images_resized)




In [6]:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)



In [12]:
base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(32, 32, 3))
base_model.trainable = False


In [8]:
model = models.Sequential([
    base_model,  # Use VGG16 as the feature extractor
    layers.Flatten(),  # Flatten the output of VGG16 to feed it to the Dense layer
    layers.Dense(64, activation='relu'),  # Add a Dense layer with 64 units and ReLU activation
    layers.Dense(10, activation='softmax')  # Output layer for classification into 10 classes (digits)
])


In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
# Train the model
history = model.fit(train_images_rgb, train_labels,
                    epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_rgb,
                                      test_labels)

print(f'Test Accuracy: {test_acc}')


Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 763s 1s/step - accuracy: 0.7395 - loss: 0.9316 - val_accuracy: 0.9281 - val_loss: 0.2479
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 841s 1s/step - accuracy: 0.9348 - loss: 0.2229 - val_accuracy: 0.9472 - val_loss: 0.1770
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 743s 991ms/step - accuracy: 0.9523 - loss: 0.1607 - val_accuracy: 0.9553 - val_loss: 0.1500
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 853s 1s/step - accuracy: 0.9604 - loss: 0.1340 - val_accuracy: 0.9603 - val_loss: 0.1262
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 792s 1s/step - accuracy: 0.9634 - loss: 0.1222 - val_accuracy: 0.9568 - val_loss: 0.1294
313/313 ━━━━━━━━━━━━━━━━━━━━ 128s 407ms/step - accuracy: 0.9589 - loss: 0.1295
Test Accuracy: 0.9617000222206116
